In [1]:
pip install --upgrade pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!cp /content/drive/MyDrive/archive_stores/*.* /content

Mounted at /content/drive
cp: cannot open '/content/drive/MyDrive/archive_stores/actors (2).gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/MyDrive/archive_stores/actors.gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/MyDrive/archive_stores/CBO2002 - Ocupacao.gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/MyDrive/archive_stores/cids_counts.gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/MyDrive/archive_stores/Cópia de df_unico4.gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/MyDrive/archive_stores/counts_ocupacoes (1).gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/MyDrive/archive_stores/counts_ocupacoes (2).gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/MyDrive/archive_stores/counts_ocupacoes.gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/MyDrive/archiv

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import lit, expr
from pyspark.sql.functions import count
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import random

In [4]:
# Criando uma SparkSession
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Exercicio Intro') \
    .getOrCreate()

###Nesta etapa, adicione código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. Carregue-o para dentro de um dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

In [23]:
df_nomes = spark.read.csv('/content/drive/MyDrive/análise de dados/nomes_aleatorios.txt')
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



###Como não informamos no momento da leitura do arquivo, o Spark não identificou o Schema por padrão e definiu todas as colunas como string. Para ver o Schema, use o método df_nomes.printSchema(). Nesta etapa, será necessário adicionar código para renomear a coluna para Nomes, imprimir o esquema e mostrar 10 linhas do dataframe.

In [24]:
df_nomes.printSchema()

root
 |-- _c0: string (nullable = true)



In [25]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
df_nomes.printSchema()

root
 |-- Nomes: string (nullable = true)



In [26]:
df_nomes.show(5)

+----------------+
|           Nomes|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [27]:
def add_coluna_randomizada(nome_coluna,lista,dataframe):
  return dataframe.withColumn(
      nome_coluna,
      F.array(
          [F.lit(x) for x in lista]
      ).getItem(
          (F.rand()*len(lista)).cast("int")
      )
  )

### Ao dataframe (df_nomes), adicione nova coluna chamada Escolaridade e atribua para cada linha um dos três valores de forma aleatória: Fundamental, Medio ou Superior.

In [28]:
escolaridade = ['Fundamental','Medio','Superior']

df_nomes = add_coluna_randomizada("Escolaridade",escolaridade,df_nomes)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/column.py:458: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [29]:
df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet|       Medio|
|    Jamie Russell|    Superior|
|   Edward Kistler|    Superior|
|    Sheila Maurer|       Medio|
| Donald Golightly|    Superior|
|       David Gray|    Superior|
|      Joy Bennett|    Superior|
|      Paul Kriese|    Superior|
|Berniece Ornellas| Fundamental|
|    Brian Farrell|       Medio|
+-----------------+------------+
only showing top 10 rows



###Ao dataframe (df_nomes), adicione nova coluna chamada Pais e atribua para cada linha o nome de um dos 13 países da América do Sul, de forma aleatória.

In [30]:
paises = ["Peru","Bolivia","Argentina","Brasil","Chile","Colômbia","Equador",
          "Guiana","Paraguai","Suriname","Uruguai","Venezuela"]

df_nomes = add_coluna_randomizada("Pais",paises,df_nomes)

In [31]:
df_nomes.show(5)

+----------------+------------+---------+
|           Nomes|Escolaridade|     Pais|
+----------------+------------+---------+
|  Frances Bennet|       Medio|  Equador|
|   Jamie Russell|    Superior| Colômbia|
|  Edward Kistler|    Superior|  Uruguai|
|   Sheila Maurer|       Medio|Venezuela|
|Donald Golightly|    Superior|    Chile|
+----------------+------------+---------+
only showing top 5 rows



### Ao dataframe (df_nomes), adicione nova coluna chamada AnoNascimento e atribua para cada linha um valor de ano entre 1945 e 2010, de forma aleatória.

In [ ]:
anos = range(1945,2010)

df_nomes = add_coluna_randomizada("AnoNascimento",anos,df_nomes)

In [33]:
df_nomes.show(5)

+----------------+------------+---------+-------------+
|           Nomes|Escolaridade|     Pais|AnoNascimento|
+----------------+------------+---------+-------------+
|  Frances Bennet|       Medio|  Equador|         1989|
|   Jamie Russell|    Superior| Colômbia|         1983|
|  Edward Kistler|    Superior|  Uruguai|         1976|
|   Sheila Maurer|       Medio|Venezuela|         1983|
|Donald Golightly|    Superior|    Chile|         1965|
+----------------+------------+---------+-------------+
only showing top 5 rows



### Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século. Armazene o resultado em outro dataframe chamado df_select e mostre 10 nomes deste.

In [34]:
# Convertendo a coluna "AnoNascimento" para tipo inteiro
df_nomes = df_nomes.withColumn("AnoNascimento", df_nomes["AnoNascimento"])

# Filtrando os registros de pessoas que nasceram neste século e armazenando o resultado em um novo dataframe
df_select = df_nomes.select("Nomes","AnoNascimento")
df_select = df_select.filter(df_select.AnoNascimento >= 2000)

df_select.show(10)

+--------------+-------------+
|         Nomes|AnoNascimento|
+--------------+-------------+
|   Frank Wiley|         2009|
| Ricky Gilbert|         2009|
| Michael Agnew|         2005|
|  James Barton|         2008|
| Jenna Johnson|         2002|
|  Jerry Remick|         2005|
|  Curtis Sutch|         2004|
|   Shelia Ceja|         2008|
|  Jason Martin|         2008|
|David Rathbone|         2004|
+--------------+-------------+
only showing top 10 rows



### Usando Spark SQL repita o processo da Pergunta 6.

In [35]:
# Criando uma tabela temporária chamada 'pessoas' para execução das consultas com Spark SQL
df_nomes.createOrReplaceTempView('pessoas')

In [36]:
# Executando consulta SQL para selecionas as pessoas que nasceram neste século
nascimento_filtrado_sql = spark.sql('SELECT * FROM pessoas WHERE AnoNascimento >= 2000')

nascimento_filtrado_sql.show(10)

+--------------+------------+---------+-------------+
|         Nomes|Escolaridade|     Pais|AnoNascimento|
+--------------+------------+---------+-------------+
|   Frank Wiley| Fundamental|Argentina|         2009|
| Ricky Gilbert| Fundamental| Suriname|         2009|
| Michael Agnew| Fundamental| Suriname|         2005|
|  James Barton|    Superior|    Chile|         2008|
| Jenna Johnson|       Medio|  Equador|         2002|
|  Jerry Remick| Fundamental|Argentina|         2005|
|  Curtis Sutch| Fundamental|   Guiana|         2004|
|   Shelia Ceja|    Superior|  Bolivia|         2008|
|  Jason Martin| Fundamental|Argentina|         2008|
|David Rathbone|       Medio|    Chile|         2004|
+--------------+------------+---------+-------------+
only showing top 10 rows



### Usando o método select do Dataframe df_nomes, Conte o número de pessoas que são da geração Millennials (nascidos entre 1980 e 1994) no Dataset

In [37]:
# Filtrando as pessoas que nasceram entre 1980 e 1994
df_millennials = df_nomes.filter((col('AnoNascimento') >= 1980) & (col('AnoNascimento') <= 1994))

# Contando o número de pessoas da geração Millennials
count_millennials = df_millennials.count()

print(f"Número de pessoas da geração Millennials: {count_millennials}")

Número de pessoas da geração Millennials: 2308848


### Repita o processo da Pergunta 8 utilizando Spark SQL

In [38]:
# Executando consulta SQL para contar o número de pessoas da geração Millennials
resultado_sql = spark.sql("SELECT COUNT(*) AS count_millennials FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994")

# Extraindo o resultado da consulta SQL
count_millennials_sql = resultado_sql.first()["count_millennials"]

print(f"Número de pessoas da geração Millennials: {count_millennials}")

Número de pessoas da geração Millennials: 2308848


In [39]:
# Criando uma função para gerar a consulta SQL de contagem por geração
def consulta_geracao(geracao,ano_inicial,ano_final):
  query = f"""
      SELECT Pais, '{geracao}' AS Geracao, COUNT(*) AS Quantidade
      FROM pessoas
      WHERE AnoNascimento BETWEEN {ano_inicial} AND {ano_final}
      GROUP BY Pais
  """
  return query

# Informações de cada geração
geracoes = [
    {"nome": "Baby Boomers", "ano_inicio": 1944, "ano_fim": 1964},
    {"nome": "Geração X", "ano_inicio": 1965, "ano_fim": 1979},
    {"nome": "Millennials", "ano_inicio": 1980, "ano_fim": 1994},
    {"nome": "Geração Z", "ano_inicio": 1995, "ano_fim": 2015}
]

# Consultando os resultados para cada geração e armazenando em uma lista
resultados_geracoes = []
for geracao_info in geracoes:
    geracao_query = consulta_geracao(geracao_info["nome"],geracao_info["ano_inicio"],geracao_info["ano_fim"])
    resultado = spark.sql(geracao_query)
    resultados_geracoes.append(resultado)

# Combinando os DataFrames usando union
df_resultado = resultados_geracoes[0]
for i in range(1, len(resultados_geracoes)):
    df_resultado = df_resultado.union(resultados_geracoes[i])

# Ordenando o resultado por Pais, Geração e Quantidade
df_resultado = df_resultado.orderBy("Pais", "Geracao", "Quantidade")

df_resultado.show()

+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    256205|
|Argentina|   Geração X|    192019|
|Argentina|   Geração Z|    192081|
|Argentina| Millennials|    191615|
|  Bolivia|Baby Boomers|    256143|
|  Bolivia|   Geração X|    192601|
|  Bolivia|   Geração Z|    191544|
|  Bolivia| Millennials|    192919|
|   Brasil|Baby Boomers|    256235|
|   Brasil|   Geração X|    191955|
|   Brasil|   Geração Z|    192212|
|   Brasil| Millennials|    193121|
|    Chile|Baby Boomers|    255974|
|    Chile|   Geração X|    192704|
|    Chile|   Geração Z|    191860|
|    Chile| Millennials|    192267|
| Colômbia|Baby Boomers|    256296|
| Colômbia|   Geração X|    192057|
| Colômbia|   Geração Z|    192439|
| Colômbia| Millennials|    191913|
+---------+------------+----------+
only showing top 20 rows

